In [1]:
import thinkdsp
import pygame,easygui
def sample_generator():
    #The Wave is Segmentet into Sequences (0.1 sec)
    GotIt = False
    while not GotIt :
        try :
            M_Name = easygui.enterbox("Enter .wav File(Name)","Music Import")
            if not ".wav" in M_Name :

                M_Name += ".wav"
            wave = thinkdsp.read_wave(M_Name)
            GotIt = True
        except:
            easygui.msgbox("Failed to load wav-File","Loading Error")
    start = 0.0
    Cash = []
    Cash2 = []
    Meng = 0
    while start < wave.duration:
        segment = wave.segment(start, 0.1)
        if segment.duration > 0:
            Meng += 1
            spectrum = segment.make_spectrum()
            Cash.append(spectrum)
            Cash2.append(segment)
        start += 0.1
    return Cash,Cash2,Meng,M_Name

In [2]:
def Render(screen,Bars,fs):
    #Peak Scanner
    SID = -1
    GR = 0
    GrID = 0
    for Bar in Bars :
            SID += 1
            if Bar[3] > GR :
                GR = Bar[3]
                GrID = SID
                Hfs = fs[GrID]
    #Main Render
    D = 5
    Len = len(Bars)
    global Epics_Cash
    ID = -1
    #Blue Line 
    if Line_Aktiv :
        for Bar in Bars :
            try :
                pygame.draw.line(screen,[0,100,100],Graph,[Bar[0]-1,Bar[3]+30],3)
                pygame.draw.line(screen,[0,100,100],[Bar[0]-1,Bar[3]+30],[Bar[0]+D+1,Bar[3]+30],3)
            except :
                pass
            Graph = [Bar[0]+D+1,Bar[3]+30] 
    #Graph
    if Bar_Aktiv :
        for Bar in Bars :
            #Bar[1]+=20
            ID += 1
            if ID == Epics_Cash[0]:
                pygame.draw.rect(screen,Epics_Cash[3],Bar)
            elif ID-1 == Epics_Cash[0] or ID+1 == Epics_Cash[0]:
                pygame.draw.rect(screen,Epics_Cash[2],Bar)
            else :
                pygame.draw.rect(screen,Epics_Cash[1],Bar)
            if ID == GrID :
                pygame.draw.rect(screen,[255,0,0],Bar)
        Epics_Cash[0] += 1
        if Epics_Cash[0] >= Len :
            Epics_Cash[0] = 0
        #pygame.draw.rect(screen,[255,255,255],[0,10,800,10],False)
    #AverageLine
    if Av_Aktiv :
        Av = 0
        Avid = 0
        for Bar in Bars :
            Avid
            Av += Bar[3]
            Avid += 1
        Av = int(Av/Avid)
        pygame.draw.line(screen,[0,100,0],[0,Av],[screen.get_width(),Av],3)
    #PeakFont
    if Pfont_Aktiv :
        font = pygame.font.SysFont("Arial",20)
        Img = font.render(str(Hfs), True,[255,255,255])
        screen.blit(Img,[700,700])

In [3]:
pygame.init()
def NextData(Cash):
    #Reurns the current Sample
    Ret = Cash[Itera[1]]
    Itera[1] += 1
    if Itera[1] == Iteralen[1]-1:
        Itera[1] = 0
    return Ret,Cash
def Update(screen,spectrum):
    #Bar ot Graph/Data
    Bars = []
    D = 0.6
    S = 1
    fs,amps = spectrum.render_full()
    ID = -1
    for PosX in fs :
        ID += 1
        if (PosX*D) < screen.get_width():
            AMPD = (amps[ID]+1)*2
            Block = 50
            if AMPD >= 800 :
                AMPD = 800
            Bars.append([PosX*D,0,5,AMPD])
    Render(screen,Bars,fs)



In [4]:
#Setup
screen = pygame.display.set_mode([1920,1080],pygame.RESIZABLE)
#VarDef
Aktiv = True
uhr = pygame.time.Clock()
Music_Name = ""
Cash,Cash2,Meng,M_Name =sample_generator()
easygui.msgbox(str(Meng)+" Samples,Sample-duration : 0.1 sec."+",length: "+str(int(Meng/10)),"Music Data")
Cash2B = Cash2[:]
CashCopy = Cash[:]
Epics_Cash = [0,[255,255,255],[200,200,200],[10,10,100]]
Itera = [0,0]
Iteralen = [len(Cash2),len(Cash)]
#Pygame Mixer Start
M_Type = "STD"
if M_Type == "STD":
    pygame.mixer.music.load(M_Name)
    pygame.mixer.music.set_volume(1)
    pygame.mixer.music.play(-1)
#Pygame setup
screen.fill([0,0,0])
pygame.key.set_repeat(80,100)
#Feature Var
Bar_Aktiv = True
Line_Aktiv = True
Av_Aktiv = True
Pfont_Aktiv = True
KeyKill = False
Runtime = 0
Restart = False

In [5]:
#Main Loop
while Aktiv :
    #Play Music/Timer
    if M_Type == "STD":
        if not Restart:
            uhr.tick(10)
            Restart = False
        Runtime = Itera[1]/10
    screen.fill([0,0,0])
    spectrum,Cash = NextData(Cash)
    if M_Type == "SAMP":
        Cash2[Itera[0]].normalize()
        Cash2[Itera[0]].apodize()
        Cash2[Itera[0]].play
        Audio(Cash2[Itera[0]].play, autoplay=True)
        Itera[0]+=1
        Runtime = Itera[1]/10
    if Itera[0] == Iteralen[0]-1:
        Itera[0] = 0
        #Restart Music
        Restart = True
    if spectrum != None:
        Update(screen,spectrum)
    else :
        Cash = CashCopy
    if Itera[1] == 0 :
        Runtime = 0
    #Runtime-Anzeige
    font = pygame.font.SysFont("Arial",20)
    Runtime2 = int(Runtime)
    Img = font.render("Runtime(Sec): "+str(Runtime2), True,[255,255,255])
    screen.blit(Img,[1000,700])
    #Main event Loop
    pygame.display.flip()
    for event in pygame.event.get():
        if event.type == pygame.QUIT :
            Aktiv = False
        elif event.type == pygame.KEYDOWN :
            #Key Handler
            if event.key == pygame.K_b :
                Bar_Aktiv = not Bar_Aktiv
            elif event.key == pygame.K_l :
                Line_Aktiv = not Line_Aktiv
            elif event.key == pygame.K_a :
                Av_Aktiv = not Av_Aktiv
            elif event.key == pygame.K_f :
                Pfont_Aktiv = not Pfont_Aktiv
            else :
                Aktiv = False
                KeyKill = True
pygame.quit()

In [6]:
if KeyKill :
    easygui.msgbox("Killed by KeyInterupt","System Shutdown")